In [1]:
from math import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_selection
from sklearn import cross_validation
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import completeness_score, homogeneity_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
%matplotlib inline

### 2. Automatic Document Clustering [Dataset: newsgroups5.zip]
For this problem you will use a different subset of the 20 Newsgroup data set that you used in Assignment 2  (see the description of the full dataset). The subset for this assignment includes 2,500 documents (newsgroup posts), each belonging to one of 5 categories windows (0), crypt (1), christian (2), hockey (3), forsale (4). The documents are represented by 9328 terms (stems). The dictionary (vocabulary) for the data set is given in the file "terms.txt" and the full term-by-document matrix is given in "matrix.txt" (comma separated values). The actual category labels for the documents are provided in the file "classes.txt". Your goal in this assignment is to perform clustering on the documents and compare the clusters to the actual categories.

Your tasks in this problem are the following [Note: for the clustering part of this assignment you should use the kMeans module form Ch. 10 of MLA (use the version provided here as it includes some corrections to the book version). You may also use Pandas and other modules from scikit-learn that you may need for preprocessing or evaluation.]

#### 2.a Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.

In [66]:
def cos_sim(obj1, obj2):
    obj1_norm = np.linalg.norm(obj1)
    obj2_norm = np.linalg.norm(obj2)
    sim = np.dot(obj2,obj1)/(obj2_norm * obj1_norm)
    dists = 1- sim
    return dists

#### 2.b Load the data set [Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [58]:
### Import Training Data, Testing Data and Labels
term_doc = pd.read_table('matrix.txt',sep=',', header = None)
terms = pd.read_table('terms.txt', header = None)
labels = pd.DataFrame(np.genfromtxt("classes.txt", delimiter=' ',dtype=int,skip_header=1))

In [59]:
### Convert term-dco to doc-term 
doc_term = term_doc.T

In [79]:
doc_term.head()

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
labels.head()

,1
0,0
1,1
2,1
3,1
4,2


In [62]:
### Remove the index
labels.drop(0, axis=1, inplace=True)
labels.head()

,1
0,0
1,1
2,1
3,1
4,2


In [63]:
### 80% - 20% split
X_train, X_test, y_train, y_test = train_test_split(doc_term, labels, test_size=0.2, random_state=50)

#### 2.c As in the case of Assignment 2, transform the term-frequencies to tfxidf values. Be sure to maintain DF values for each of the terms in the dictionary. [Note: if you run into problems due to limited computational resources, you may prune the data by removing all terms with low DF values, e.g., terms that appear in less than 10 documents. Be sure to maintain the correspondence between the dictionary terms and the matrix rows.]

In [52]:
X_train_T = X_train.T
X_test_T = X_test.T

### Store some values
NTerm = X_train.shape[1]
NDoc_train = X_train.shape[0]
NDoc_test = X_test.shape[0]

### Document Frequency
DF = np.array([(X_train_T!=0).sum(1)]).T

### IDF
train_Mat = np.ones(np.shape(X_train_T), dtype=float)*NDoc_train
test_Mat = np.ones(np.shape(X_test_T), dtype=float)*NDoc_train
TrainIDF = np.log2(np.divide(train_Mat, DF), dtype=float)
TestIDF = np.log2(np.divide(test_Mat, DF), dtype=float)

### TF*IDF(use the same idf from training set)
TF_IDF_train= np.array(X_train_T * TrainIDF, dtype=float)
TF_IDF_test = np.array(X_test_T * TestIDF, dtype=float)

TF_IDF_train = TF_IDF_train.T
TF_IDF_test = TF_IDF_test.T

In [53]:
### check the size
print(TF_IDF_train.shape, TF_IDF_test.shape)

(2000, 9328) (500, 9328)


In [54]:
### Got errors from 2.d: Need to take care of NaNs and infinite values
def cleaning(df):
    df[np.isnan(df)] = 0
    df[df == inf] = 0
    df[df == -inf] = 0
    return df
TF_IDF_train = cleaning(TF_IDF_train)
TF_IDF_test = cleaning(TF_IDF_test)

#### 2.d Perform Kmeans clustering on the training data. Write a function to display the top N terms in each cluster along with the cluster DF values for each term, percentage of docs in the cluster in which the terms appear, and the size of the cluster. Sort the terms in decreasing order of the DF percentage. [Extra Credit: use your favorite third party tool, ideally with a Python based API, to create a word cloud for each cluster based on the in-cluster DF values.]

##### Copy the K-means scripts here (import doesn't work for python 3)

In [67]:
'''
Created on Feb 16, 2011
k Means Clustering for Ch10 of Machine Learning in Action
@author: Peter Harrington
'''
from numpy import *

def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = zeros((k,n), dtype=float)
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k)
        
    return centroids 

def kMeans(dataSet, k, distMeas=cos_sim, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = zeros((m,2))
    #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2

        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment

def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
    centList =[centroid0] #create a list with one centroid
    for j in range(m): #calc initial Error
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:] #get the data points currently in cluster i
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0] == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[nonzero(bestClustAss[:,0] == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids 
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return mat(centList), clusterAssment


In [69]:
centroids, clusters = kMeans(mat(TF_IDF_train), 5, distMeas=cos_sim)

In [72]:
### Get help from other students for the zip function
def top_terms(df, k, N_term):

    for i in range(k):
        ### First, print out cluster number and the number of obs in it
        n_cluster = df[clusters[:,0]==i]
        n_obs = n_cluster.shape[0]
        print('Cluster: ',i)
        print("cluster %d has %d observations" % (i, n_obs))
        
        ### Calculate document frequency in each cluster        
        doc_freq_cluster = np.array([(n_cluster.T!=0).sum(1)]).T
        
        ### Calculate the percentage of DF
        pc_doc_freq_cluster = doc_freq_cluster/float(n_cluster.shape[0])
        pc_doc_freq_cluster = map(list, pc_doc_freq_cluster)
        pc_doc_freq_cluster = [i[0] for i in pc_doc_freq_cluster]
        cluster_DF = [i[0] for i in doc_freq_cluster]
        
        ### Zip them together
        sort=sorted(zip(terms, doc_freq_cluster, pc_doc_freq_cluster),key=lambda x:x[2],reverse=True)
        for i in sort[:N_term]:
               print(i[0], '\t', i[1], '\t', i[2])
        print('\n')
       
    return centroids, clusters

In [73]:
centroids, clusters = top_terms(np.array(TF_IDF_train), 5, 5)

cluster 0 :
cluster 0  size:  402
term 	 frequency 	 PercentOfDocs
['subject'] 	 [402] 	 1.0
['write'] 	 [247] 	 0.614427860697
['god'] 	 [223] 	 0.554726368159
['on'] 	 [220] 	 0.547263681592
['christian'] 	 [196] 	 0.487562189055


cluster 1 :
cluster 1  size:  394
term 	 frequency 	 PercentOfDocs
['subject'] 	 [394] 	 1.0
['write'] 	 [221] 	 0.560913705584
['game'] 	 [210] 	 0.532994923858
['team'] 	 [183] 	 0.464467005076
['go'] 	 [176] 	 0.446700507614


cluster 2 :
cluster 2  size:  395
term 	 frequency 	 PercentOfDocs
['subject'] 	 [395] 	 1.0
['window'] 	 [278] 	 0.703797468354
['write'] 	 [176] 	 0.445569620253
['file'] 	 [149] 	 0.377215189873
['get'] 	 [139] 	 0.351898734177


cluster 3 :
cluster 3  size:  424
term 	 frequency 	 PercentOfDocs
['subject'] 	 [424] 	 1.0
['sale'] 	 [237] 	 0.558962264151
['email'] 	 [165] 	 0.389150943396
['pleas'] 	 [154] 	 0.36320754717
['offer'] 	 [116] 	 0.27358490566


cluster 4 :
cluster 4  size:  385
term 	 frequency 	 PercentOfDocs
['su

#### 2.e Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [86]:
cluster[:,0]

array([ 4.,  1.,  3., ...,  3.,  1.,  3.])

In [90]:
com = completeness_score(np.array(y_train.T)[0],cluster[:,0])
hom = homogeneity_score(np.array(y_train.T)[0],cluster[:,0])
print ("Completeness is %0.001f" % (com))
print ("Homogeneity is %0.001f" % (hom))

Completeness is 0.8
Homogeneity is 0.8
